# Lecture: 9 Must‑Know Pandas Operations for Working with Data

This notebook is a step‑by‑step lecture. Each concept includes:
1) a short explanation,
2) a concrete example, and
3) what to look for in the output.

All examples are self‑contained and use small, accessible datasets. Run cells in order.


## Setup

We import pandas and numpy, and set display options so tables are easier to read.


In [ ]:
import pandas as pd
import numpy as np

pd.set_option("display.max_columns", None)
pd.set_option("display.width", 120)
pd.__version__


## 1. Data Import

We demonstrate common ways to load data: CSV, Excel (via a generated file), JSON, SQL-like (using sqlite), and Parquet.
The examples ensure you can run them offline by creating small files on the fly.

Look for: each load method returns a DataFrame with the expected columns and rows.


In [ ]:
# CSV: create and read
df_csv_src = pd.DataFrame({'id':[1,2,3], 'name':['Ada','Bob','Cai'], 'score':[88,91,79]})
csv_path = 'example.csv'
df_csv_src.to_csv(csv_path, index=False)
df_csv = pd.read_csv(csv_path)
df_csv


In [ ]:
# Excel: create and read from a specific sheet
xlsx_path = 'example.xlsx'
with pd.ExcelWriter(xlsx_path) as writer:
    df_csv_src.to_excel(writer, index=False, sheet_name='Sheet1')
    (df_csv_src.assign(score=lambda d: d['score']+1)).to_excel(writer, index=False, sheet_name='Sheet2')

df_xlsx = pd.read_excel(xlsx_path, sheet_name='Sheet1')
df_xlsx


In [ ]:
# JSON: read from a JSON string/file
data_json = [
    {"id": 1, "dept": "A", "active": True},
    {"id": 2, "dept": "B", "active": False}
]
json_path = 'example.json'
pd.Series(data_json).to_json(json_path, orient='values')
df_json = pd.read_json(json_path)
df_json


In [ ]:
# SQL: use sqlite to simulate read_sql
import sqlite3
conn = sqlite3.connect(':memory:')
pd.DataFrame({'x':[10,20,30],'y':['p','q','r']}).to_sql('tbl', conn, index=False, if_exists='replace')
df_sql = pd.read_sql('select * from tbl where x >= 20', conn)
df_sql


In [ ]:
# Parquet: write and read a parquet file (needs pyarrow or fastparquet; we fall back to CSV if unavailable)
parquet_path = 'example.parquet'
try:
    df_csv_src.to_parquet(parquet_path)
    df_parquet = pd.read_parquet(parquet_path)
except Exception as e:
    df_parquet = pd.read_csv(csv_path)  # fallback for environments without parquet engine
df_parquet


## 2. Data Selection

We practice selecting columns and rows using `[]`, `.loc` (label-based), `.iloc` (integer-based), boolean filters,
`query`, and membership with `isin`.

Example dataset: a small customer table.
Look for: how each selection returns either a Series or a DataFrame with the intended subset.


In [ ]:
customers = pd.DataFrame({
    'customer_id':[101,102,103,104,105],
    'name':['Alice','Bruno','Chin','Diana','Evan'],
    'age':[25,41,33,29,52],
    'city':['NY','SF','LA','NY','LA'],
    'spend':[120.5, 340.0, 85.2, 220.0, 560.0]
})
customers


In [ ]:
# Single column (Series)
customers['age'].head()


In [ ]:
# Label-based selection with .loc: choose rows by labels and columns by names
customers.loc[1:3, ['name','city','spend']]


In [ ]:
# Integer-based selection with .iloc: choose rows and columns by integer positions
customers.iloc[0:3, 0:3]


In [ ]:
# Boolean filtering
customers[customers['spend'] > 200][['name','spend']]


In [ ]:
# SQL-like filtering with .query
customers.query('age >= 30 and city == "LA"')


In [ ]:
# Multiple values with .isin
customers[customers['city'].isin(['NY','LA'])]


## 3. Data Manipulation

We demonstrate grouping, merging, pivot tables, sorting, melting (unpivot), and applying functions.

Example: two sales tables for merging and a line-item table for pivot/melt.
Look for: how groupby aggregates per key, how merges combine columns, and how pivot/melt reshape the data.


In [ ]:
orders = pd.DataFrame({
    'order_id':[1,2,3,4,5],
    'customer_id':[101,101,102,104,105],
    'amount':[120,80,340,220,560],
    'channel':['web','store','web','web','store']
})
customers_small = customers[['customer_id','name','city']]
orders, customers_small.head()


In [ ]:
# Groupby with multiple aggregations
orders.groupby('customer_id').agg(total_amount=('amount','sum'),
                                  avg_amount=('amount','mean'),
                                  n_orders=('order_id','count')).reset_index()


In [ ]:
# Merge: left-join orders with customers to enrich records
orders_enriched = orders.merge(customers_small, on='customer_id', how='left')
orders_enriched


In [ ]:
# Pivot table: sales by city and channel
pivot = pd.pivot_table(orders_enriched, values='amount', index='city', columns='channel', aggfunc='sum', fill_value=0)
pivot


In [ ]:
# Sort values by amount then by channel
orders.sort_values(['amount','channel'], ascending=[False, True])


In [ ]:
# Melt (unpivot): turn wide pivot back to long format
pivot_reset = pivot.reset_index()
melted = pivot_reset.melt(id_vars='city', value_vars=pivot.columns.tolist(), var_name='channel', value_name='amount')
melted.sort_values(['city','channel']).head()


In [ ]:
# Apply: vectorized transformation of a column
orders.assign(amount_with_tax=lambda d: d['amount'] * 1.07)


## 4. Data Cleaning

We cover dropping missing values, forward-filling, removing duplicates, replacing values, casting types, and interpolation.

Example dataset includes intentional issues: missing values, duplicates, and inconsistent coding.
Look for: post-cleaning outputs have no duplicates, reasonable types, and filled values where appropriate.


In [ ]:
raw = pd.DataFrame({
    'id':[1,2,2,3,4,5],
    'category':['old','new','new','old',None,'old'],
    'value':[10,np.nan,20,15,30,np.nan]
})
raw


In [ ]:
# Drop rows with any missing in selected columns
dropped = raw.dropna(subset=['category'], how='any')
dropped


In [ ]:
# Forward fill missing values in 'value'
ffill = raw.sort_values('id').copy()
ffill['value'] = ffill['value'].fillna(method='ffill')
ffill


In [ ]:
# Remove duplicated ids (keep first)
no_dupes = raw.drop_duplicates(subset=['id'])
no_dupes


In [ ]:
# Replace values
replaced = raw.copy()
replaced['category'] = replaced['category'].replace({'old':'legacy'})
replaced


In [ ]:
# Cast types
typed = raw.copy()
typed['id'] = typed['id'].astype('int64')
typed.dtypes


In [ ]:
# Interpolate numeric series linearly
interpolated = raw.copy()
interpolated['value'] = interpolated['value'].interpolate(method='linear')
interpolated


## 5. String Operations

We demonstrate `str.contains`, `str.extract`, `str.split`, `str.lower`, `str.strip`, and `str.replace`.

Example: product codes encoded in text.
Look for: how vectorized `.str` methods operate on entire columns.


In [ ]:
products = pd.DataFrame({
    'sku':['A-100-NY','B-250-SF','C-010-LA','B-333-NY','A-120-LA'],
    'desc':['Widget Alpha','Bolt Pack 250','Cable 10ft','Bolt Pack 333','Widget Alpha Pro']
})
products


In [ ]:
# str.contains: find rows describing Bolt
products[products['desc'].str.contains('Bolt', case=False)]


In [ ]:
# str.extract: capture the numeric part in sku
products['qty'] = products['sku'].str.extract(r'-(\d+)-')
products


In [ ]:
# str.split: split into parts and select the city part
products['city'] = products['sku'].str.split('-').str[2]
products[['sku','city']]


In [ ]:
# str.lower and str.strip
products['norm_desc'] = products['desc'].str.lower().str.strip()
products[['desc','norm_desc']]


In [ ]:
# str.replace: remove non-alphanumerics from desc
products['desc_simple'] = products['desc'].str.replace(r'[^0-9a-zA-Z ]+','', regex=True)
products[['desc','desc_simple']]


## 6. Statistics

We compute summary statistics, per-column aggregations, value counts, correlations, covariance, and quantiles.

Example: reuse the `orders_enriched` table.
Look for: distribution summaries and relationships between numeric variables.


In [ ]:
# Summary statistics
orders_enriched[['amount']].describe()


In [ ]:
# Per-column aggregation with agg on a column
orders_enriched['amount'].agg(['mean','median','std'])


In [ ]:
# Value counts (normalized share of orders by channel)
orders_enriched['channel'].value_counts(normalize=True)


In [ ]:
# Correlation matrix among numeric columns
orders_enriched[['amount','customer_id']].corr(method='pearson')


In [ ]:
# Covariance matrix
orders_enriched[['amount','customer_id']].cov()


In [ ]:
# Quantiles
orders_enriched['amount'].quantile([0.25, 0.5, 0.75])


## 7. Time Series

We construct a realistic daily sales time series with seasonality and noise to demonstrate resampling, rolling windows,
shifts, date_range creation, frequency conversion with `asfreq`, and datetime formatting with `dt.strftime`.

Look for: monthly resampling aggregates, moving averages smoothing fluctuations, and shifted series aligning past values.


In [ ]:
rng = pd.date_range('2024-01-01', periods=365, freq='D')
rng.name = 'date'
np.random.seed(7)

base = 200 + 20*np.sin(2*np.pi*(rng.dayofyear)/7)  # weekly seasonality
trend = np.linspace(0, 50, len(rng))               # upward trend across the year
noise = np.random.normal(0, 10, len(rng))
sales = np.maximum(0, base + trend + noise)

ts = pd.DataFrame({'sales': sales}, index=rng)
ts.head()


In [ ]:
# Resample to monthly average sales
monthly_avg = ts['sales'].resample('M').mean()
monthly_avg.head()


In [ ]:
# Rolling 7-day mean to smooth noise
ts['rolling_7d'] = ts['sales'].rolling(window=7).mean()
ts[['sales','rolling_7d']].head(12)


In [ ]:
# Shift the series by one period to compare to "yesterday"
ts['sales_lag1'] = ts['sales'].shift(periods=1)
ts[['sales','sales_lag1']].head(5)


In [ ]:
# Create a custom date_range: first business day of each month in 2024
custom_range = pd.date_range('2024-01-01', periods=12, freq='BMS')
custom_range


In [ ]:
# asfreq to insert explicit daily frequency with forward fill of missing (illustration by dropping dates then restoring)
ts_thin = ts.iloc[::3].copy()         # keep every 3rd day
ts_ffill = ts_thin.asfreq('D').ffill()
ts_ffill.head(10)


In [ ]:
# Datetime formatting with dt.strftime
date_labels = ts.index.to_series().dt.strftime('%Y-%m-%d')
date_labels.head()


## 8. Advanced Features

We show `pipe` for method chaining, `eval` for expression evaluation, `memory_usage`, `select_dtypes`, `nlargest`,
and `explode` for list-like columns.

Look for: cleaner pipelines with `pipe`, selections by dtype, and expansion of list columns with `explode`.


In [ ]:
# pipe: define a function and insert into a chain
def add_net_amount(df, tax_rate=0.07):
    return df.assign(net_amount=lambda d: d['amount']*(1+tax_rate))

chained = (orders_enriched
           .pipe(add_net_amount, tax_rate=0.05)
           .sort_values('net_amount', ascending=False)
           .head(3))
chained


In [ ]:
# eval: compute an expression over columns
orders_enriched.eval('double_amount = amount * 2', inplace=False)


In [ ]:
# memory_usage: inspect approximate memory footprint
orders_enriched.memory_usage(deep=True).sum()


In [ ]:
# select_dtypes: pick numeric columns only
orders_enriched.select_dtypes(include=['number']).head()


In [ ]:
# nlargest: top-N orders by amount
orders_enriched.nlargest(3, 'amount')


In [ ]:
# explode: expand list-like column to long rows
multi = pd.DataFrame({
    'order_id':[10,11],
    'items':[ ['A','B','C'], ['B'] ]
})
multi_exploded = multi.explode('items', ignore_index=True)
multi_exploded


## 9. Data Export

We demonstrate exporting to CSV, Excel, JSON, and Parquet. Files are written to the current folder.
Open them after running if you'd like to inspect contents.

Look for: files created with the expected shape and fields.


In [ ]:
orders_enriched.to_csv('orders_out.csv', index=False)
with pd.ExcelWriter('orders_out.xlsx') as w:
    orders_enriched.to_excel(w, index=False, sheet_name='Sheet1')
orders_enriched.to_json('orders_out.json', orient='records')

try:
    orders_enriched.to_parquet('orders_out.parquet')
    export_status = 'CSV, Excel, JSON, Parquet exported.'
except Exception:
    export_status = 'CSV, Excel, JSON exported. Parquet skipped (engine unavailable).'
export_status


## Tips and Best Practices

1. Use `.copy()` when creating DataFrame views that you plan to modify to avoid chained assignment issues.
2. Prefer method chaining with `pipe` and temporary variables kept to a minimum for readability.
3. For categorical data, set `dtype='category'` to save memory and enable category-aware operations.
4. Use `inplace=True` sparingly. It often returns `None` and can make pipelines harder to reason about.
5. Validate assumptions early with `df.sample()`, `df.info()`, and `.describe()` before heavy transformations.
